# Analysis

So what did I mean when saying I'd explore the differences between speeches and songs? That's a good question, especially since each corpus was authored by a different group. 

First, we'll start off with some simple features of the corpora such as vocabulary, length, word frequency (in-document) and any more that come to mind as I progress. Following that, we'll dive into sentiment analysis and topic modeling, see if songs are more subjective than speeches or if "great speeches" are often positive. Remember this is a learning project, the data being explored may be a bit silly but through it we can learn different NLP techniques and see the problems associated with real life data.

In [1]:
import numpy as np
import pandas as pd
import logging

In [2]:
comm_df = pd.read_excel('communication_data.xlsx')
bow_df = pd.read_excel('communication_data_bow.xlsx')

In [5]:
comm_df['Chars'] = comm_df['Corpus'].apply(len)
comm_df['Words'] = comm_df['Corpus'].apply(lambda c: len(c.split()))
comm_df['Unique Words'] = bow_df['Corpus'].apply(lambda c: len(set(c.split())))
comm_df

,Originator,Title,Corpus,Source,Chars,Words,Unique Words
0,Chimamanda Ngozi Adichie,The Danger of a Single Story,I'm a storyteller. And I would like to tell yo...,https://jamesclear.com/great-speeches/the-dang...,15842,2845,853
1,Jeff Bezos,What Matters More Than Your Talents,"As a kid, I spent my summers with my grandpare...",https://jamesclear.com/great-speeches/what-mat...,7231,1353,490
2,John C. Bogle,Enough,Here’s how I recall the wonderful story that s...,https://jamesclear.com/great-speeches/enough-b...,8459,1428,542
3,Brené Brown,The Anatomy of Trust,"Oh, it just feels like an incredible understat...",https://jamesclear.com/great-speeches/the-anat...,4681,903,282
4,John Cleese,Creativity in Management,"You know, when Video Arts asked me if I'd like...",https://jamesclear.com/great-speeches/creativi...,27576,4889,1280
5,William Deresiewicz,Solitude and Leadership,My title must seem like a contradiction. What ...,https://jamesclear.com/great-speeches/solitude...,32767,5733,1543
6,Richard Feynman,Seeking New Laws,What I want to talk to you about tonight is st...,https://jamesclear.com/great-speeches/seeking-...,32767,5777,1180
7,Neil Gaiman,Make Good Art,I never really expected to find myself giving ...,https://jamesclear.com/great-speeches/make-goo...,15402,2918,785
8,John W. Gardner,Personal Renewal,I'm going to talk about “Self-Renewal.” One of...,https://jamesclear.com/great-speeches/personal...,22673,4110,1209
9,Elizabeth Gilbert,Your Elusive Creative Genius,I am a writer. Writing books is my profession ...,https://jamesclear.com/great-speeches/your-elu...,18680,3459,931
